In [1]:
from bs4 import BeautifulSoup
import urllib.parse
import requests
import pickle
import collections
from tqdm import tqdm
import time
import re
import random

In [2]:
# Create lists
plants_dict = pickle.load(open('../../data/processed/train_dataPOWO.pkl', 'rb'))
birds_dict = pickle.load(open('../../data/processed/train_dataBOW.pkl', 'rb'))

plants_list = [keys for keys, values in plants_dict.items()]
birds_list = [keys for keys, values in birds_dict.items()]

In [5]:
def search_Duck(query):
    
    # Get results 
    page = requests.get('https://duckduckgo.com/html/?q={0}'.format(query), 
                        headers={'user-agent': 'Descriptor/0.0.1'})
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('a', attrs={'class':'result__a'}, href=True)
    
    # Init list
    links = []
    # Clean results
    for link in results:
        url = link['href']
        o = urllib.parse.urlparse(url)
        d = urllib.parse.parse_qs(o.query)
        links.append(d['uddg'][0])
        
    return links

def search_Bing(query):
    
    # Get results
    page = requests.get('https://www.bing.com/search?form=MOZLBR&pc=MOZI&q={0}'.format(query), 
                        headers={'user-agent': 'Descriptor/0.0.1'})
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Init list
    links = [] 
    # Clean results
    for i in soup.find_all('a', attrs={'h':re.compile('ID=SERP.+')}, href=True):
        link = i['href']
        if link.startswith('http') and 'microsoft' not in link and 'bing' not in link:
            links.append(link)
            
    return links

In [6]:
# Init dict
search_links = collections.defaultdict(list)

# Query
query = 'description'

# Loop over data
for count, plant in enumerate(tqdm(plants_list[0:20])):
        
    # create q
    plant_q = plant.replace(' ', '+')
    plant_q += '+' + query
    
    if count % 2:
        search_links[plant] += search_Duck(plant_q)
    else:
        search_links[plant] += search_Bing(plant_q)
    
    time.sleep(1.5)
        
    

100%|███████████████████████████████████████████| 20/20 [00:39<00:00,  1.97s/it]


In [7]:
with open('combi_test.pkl', 'wb') as f:
    pickle.dump(search_links, f)    

In [8]:
search_links

defaultdict(list,
            {'Hirtella': ['https://en.wikipedia.org/wiki/Hirtella#:~:text=Hirtella%20is%20a%20genus%20of%20110%20species%20of,Madagascar.%20The%20flowers%20are%20mainly%20pollinated%20by%20butterflies.',
              'https://www.merriam-webster.com/dictionary/Hirtella',
              'https://en.wikipedia.org/wiki/Hirtella',
              'https://en.wikipedia.org/wiki/Hirtella',
              'https://en.wikipedia.org/wiki/Hirtella',
              'http://creativecommons.org/licenses/by-sa/3.0/',
              'https://www.merriam-webster.com/dictionary/Hirtella',
              'https://en.wikipedia.org/wiki/Melanogaster_hirtella',
              'https://en.wikipedia.org/wiki/Melanogaster_hirtella',
              'http://creativecommons.org/licenses/by-sa/3.0/',
              'https://bioportal.naturalis.nl/taxon/Hirtella_americana?language=en',
              'https://en.wikipedia.org/wiki/Asclepias_hirtella'],
             'Murdannia': ['https://en.wikipedia.org/w